In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import requests
from pandas.io.json import json_normalize

### Creation of previous dataframe condensed into single cell:

In [2]:
# Import html into pandas dataframe and Select required table into dataframe variable (df):
alltables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = alltables[0]

# Ignore cells with a borough that is **Not assigned**:
df.drop(df[df['Borough'] == 'Not assigned'].index, axis = 0, inplace = True)

# If a cell has a borough but a **Not assigned** neighborhood, then the *neighborhood* will be the same as the *borough*.
df['Neighborhood'].replace(to_replace = 'Not assigned', value = df['Borough'], inplace = True)

# **Dataframe cleanup** (neighborhoods separated by comma, rename column and index reset):
df['Neighborhood']=df['Neighborhood'].apply(lambda x: x.replace(' /', ','))
df.rename(columns={"Postal code": "Postal Code"}, inplace=True)
df.reset_index(inplace=True, drop = True)

df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Import Geospatial Coordinates:

In [3]:
geo = pd.read_csv('Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge Dataframes

In [4]:
df = df.merge(geo, on=['Postal Code'], how='left')

### Check Updated Dataframe

In [5]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [6]:
df.shape

(103, 5)

In [7]:
df.isna().sum()

Postal Code     0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [8]:
# df['Postal Code']=df['Postal Code'].astype('str')
# geo['Postal Code']=geo['Postal Code'].astype('str')

In [9]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


### Get coordinates of Toronto City

In [10]:
address = 'Toronto City'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### Toronto City Map

In [11]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Neighborhoods containing the work 'York'

In [12]:
york = df[df['Borough'].astype(str).str.contains('York')]
york.reset_index(inplace=True, drop = True)
york

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
5,M6B,North York,Glencairn,43.709577,-79.445073
6,M3C,North York,Don Mills,43.725900,-79.340923
7,M4C,East York,Woodbine Heights,43.695344,-79.318389
8,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512


In [13]:
york.shape

(34, 5)

In [14]:
# create map of York dataframe using latitude and longitude values
map_york = folium.Map(location=[latitude+0.06, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york['Latitude'], york['Longitude'], york['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

In [15]:
CLIENT_ID = 'AISHYS45K1UNBO4BMDGMKGWQVNVNXBPKEAQEM4FCQEW2DEW1' # your Foursquare ID
CLIENT_SECRET = '04LIUNELIOIHAOU3V1XC3GFRRS1LGBJ1MFESWONURQVC3SI2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AISHYS45K1UNBO4BMDGMKGWQVNVNXBPKEAQEM4FCQEW2DEW1
CLIENT_SECRET:04LIUNELIOIHAOU3V1XC3GFRRS1LGBJ1MFESWONURQVC3SI2


**Analysis of attractions in or around Humewood-Cedarvale neighborhood**:

In [16]:
neighborhood_latitude = york.loc[8, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = york.loc[8, 'Longitude'] # neighborhood longitude value
neighborhood_name = york.loc[8, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Humewood-Cedarvale are 43.6937813, -79.42819140000002.


In [17]:
# type your answer here
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb0481e0f5968001b764da4'},
 'response': {'headerLocation': 'Cedarvale',
  'headerFullLocation': 'Cedarvale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6982813045, 'lng': -79.4219793104081},
   'sw': {'lat': 43.689281295499995, 'lng': -79.43440348959193}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b0afc19f964a520212b23e3',
       'name': 'Cedarvale Park',
       'location': {'address': '100 glen cedar',
        'crossStreet': 'Strathearn',
        'lat': 43.692534923091934,
        'lng': -79.42870527613704,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.692534923091934,
          'lng': -79.42870527613704}],
        'distance': 144,


In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/Users/venkat/opt/anaconda3/envs/ml_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Cedarvale Park,Field,43.692535,-79.428705
1,Cedarvale Ravine,Trail,43.690188,-79.426106
2,Cedarvale Dog Park,Dog Run,43.692036,-79.429491
3,Phil White Arena,Hockey Arena,43.691303,-79.431761


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [21]:
# create map of York dataframe using latitude and longitude values
map_ngbr = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15.5)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='brown',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ngbr)  
    
map_ngbr